# The Battle of Neighborhoods

## Toronto and New York City

# Introduction

Toronto is the capital city of the Canadian province of Ontario. With a recorded population of 2,731,571 in 2016, it is the most populous city in Canada and the fourth most populous city in North America. 

New York City is the most populous city in the United States. With an estimated 2019 population of 8,336,817 distributed over about 302.6 square miles, New York City is also the most densely populated major city in the United States.

Toronto and New York City are the popular vacation and migration destinations for people all around the world. They are diverse and multicultural and offer a wide variety of experiences that is widely sought after.  We try to group the neighborhoods of Toronto and New York City respectively and draw insights to what they look like now.

# Business Problem

The aim is to help people choose their destinations depending on the experiences that the neighborhoods have to offer and what they would want to have. This also helps people make decisions if they are thinking about migrating to Toronto or New York City or even if they want to relocate neighborhoods within the city. Our findings will help stakeholders make informed decisions and address any concerns they have including the different kinds of cuisines, provision stores and what the city has to offer.

# Data Description

We require geolocation data for both Toronto and New York City. Postal codes in each city serve as a starting point. Using Postal codes, we use can find out the neighborhoods, boroughs, venues and their most popular venue categories.

## Toronto

To derive our solution, we scrape our data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

This Wikipedia page has information about all the neighborhoods, we limit it Toronto.

Also, it lacks of information about the geographical locations. To solve this problem, we use the Geocoder Python package.


## New York City

To derive our solution, we leverage JSON data available at https://geo.nyu.edu/catalog/nyu_2451_34572

## Foursquare API Data

We will need data about different venues in different neighborhoods of that specific borough. In order to gain that information, we will use "Foursquare" locational information. Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos. As such, the foursquare location platform will be used as the sole data source since all the stated required information can be obtained through the API.

After finding the list of neighborhoods, we then connect to the Foursquare API to gather information about venues inside every neighborhood. For each neighborhood, we have chosen the radius to be 500 meters.



The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:

* *Neighborhood*: Name of the Neighborhood
* *Neighborhood Latitude*: Latitude of the Neighborhood
* *Neighborhood Longitude*: Longitude of the Neighborhood
* *Venue*: Name of the Venue
* *Venue Latitude*: Latitude of Venue
* *Venue Longitude*: Longitude of Venue
* *Venue Category*: Category of Venue


Based on all the information collected for both Toronto and New York City, we have sufficient data to build our model. We cluster the neighborhoods together based on similar venue categories. We then present our observations and findings. Using this data, our stakeholders can take the necessary decision.

# Methodology

We will be creating our model with the help of Python so we start off by importing all the required packages.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0
import folium

from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



Package breakdown:

* *Pandas* : To collect and manipulate data in JSON and HTMl and then data analysis
* *requests* : Handle http requests
* *matplotlib* : Detailing the generated maps
* *folium* : Generating maps of Toronto and New York City
* *sklearn* : To import Kmeans which is the machine learning model that we are using

The approach taken here is to explore each of the cities individually, plot the map to show the neighborhoods being considered and then build our model by clustering all the similar neighborhoods together and finally plot the new map with the clustered neighborhoods. We draw insights and then compare and discuss our findings.

# Exploring Toronto

## Neighborhoods of Toronto

We begin to start collecting and refining the data needed for the our business solution to work.

### Data Collection

To get the neighborhoods in Toronto, we start by scraping the list of postal codes of Canada Wikipedia page.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data  = requests.get(url).text

In [3]:
soup = BeautifulSoup(html_data,"html.parser")
soup.find_all('title')

[<title>List of postal codes of Canada: M - Wikipedia</title>]

In [4]:
table_contents = pd.DataFrame(columns=['Postal Code', 'Borough', 'Neighborhood'])

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':  #To skip if the borough name is 'Not Assigned'
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)    

In [5]:
df=pd.DataFrame(table_contents)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Data Preprocessing

In [6]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Geolocations of the Toronto Neighborhoods

In [7]:
data = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


#### Since the dimensions of both dataframes are the same, we can join on postal code

#### Check the column types of both dataframes

In [8]:
data.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [9]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

#### Join the two dataframes

In [10]:
combined_data = df.join(data.set_index('Postal Code'), on='PostalCode', how='inner')
combined_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### Use geopy library to get the latitude and longitude values of Toronto

In [11]:
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


## Visualize the Map of Toronto

To help visualize the Map of Toronto and the neighborhoods in Toronto, we make use of the folium package.

In [12]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighborhood in zip(combined_data['Latitude'], combined_data['Longitude'], combined_data['Borough'], combined_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Toronto)  
    
map_Toronto

## Explore Neighborhoods in Toronto

Using Foursquare API, we are able to get the venue and venue categories around each neighborhood in Toronto.

#### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'GDVLLEAOEYDPMLLND1PEIFP2UKLQ3TC42WEVXMSGCHVLT23S' 
CLIENT_SECRET = 'BV0P1UO5OLCHN0AOLTSAI2Q4SQQJEPKSC3TZK2E4GZXWNB2W'
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GDVLLEAOEYDPMLLND1PEIFP2UKLQ3TC42WEVXMSGCHVLT23S
CLIENT_SECRET:BV0P1UO5OLCHN0AOLTSAI2Q4SQQJEPKSC3TZK2E4GZXWNB2W


#### Create a function to get all the venue categories in Toronto

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Collect the venues in Toronto for each neighborhood

In [15]:
venues_in_toronto = getNearbyVenues(combined_data['Neighborhood'], combined_data['Latitude'], combined_data['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [16]:
venues_in_toronto.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [17]:
venues_in_toronto.shape

(2007, 7)

We have scraped total of 2007 records for venues.

#### Grouping by Neighborhoods

In [18]:
venues_in_toronto.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",15,15,15,15,15,15
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
Willowdale West,5,5,5,5,5,5
"Willowdale, Newtonbrook",1,1,1,1,1,1
Woburn,4,4,4,4,4,4


In [19]:
print('There are {} uniques categories.'.format(len(venues_in_toronto['Venue Category'].unique())))

There are 259 uniques categories.


### Analyze Each Neighborhood

#### One Hot Encoding

We need to Encode our venue categories to get a better result for our clustering.

In [20]:
# one hot encoding
Toronto_onehot = pd.get_dummies(venues_in_toronto[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = venues_in_toronto['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Venue categories mean value

We will group the neighborhoods and calculate the mean of the frequency of occurrence of each category.

In [21]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
96,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0


#### Create a function to get the top most common venue categories

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

There are too many venue categories, we can take the top 10 to cluster the neighborhoods.

Creating a function to label the columns of the venue correctly.

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

### Top venue categories

Getting the top venue categories in Toronto

In [24]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Latin American Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
1,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place,Playground,Dance Studio,Coffee Shop,Skating Rink,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Sushi Restaurant,Diner,Gas Station,Sandwich Place,Frozen Yogurt Shop,Ice Cream Shop,Bridal Shop
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Museum,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
4,"Bedford Park, Lawrence Manor East",Restaurant,Italian Restaurant,Coffee Shop,Sandwich Place,Indian Restaurant,Sushi Restaurant,Café,Liquor Store,Butcher,Thai Restaurant


## Model Building

### K-Means

Let's cluster the city of Toronto to roughly 5 to make it easier to analyze.

We use the K Means clustering technique to do so.

In [25]:
# Run k-means to cluster the neighborhood into 5 clusters

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 4], dtype=int32)

### Labelling Clustered Data

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = combined_data

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Fast Food Restaurant,Park,Food & Drink Shop,Martial Arts School,Malay Restaurant,Medical Center,Mediterranean Restaurant,Men's Store,Moroccan Restaurant,Metro Station
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Nail Salon,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Park,Pub,Bakery,Café,Restaurant,Farmers Market,Sushi Restaurant,Beer Store,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Miscellaneous Shop,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Salad Place,Mexican Restaurant,Burger Joint,Café,Sandwich Place,Restaurant


#### Drop all the NaN values to prevent data skew

In [27]:
Toronto_merged_nonan = Toronto_merged.dropna(subset=['Cluster Labels'])

### Visualizing the Clustered neighborhood

In [28]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged_nonan['Latitude'], Toronto_merged_nonan['Longitude'], Toronto_merged_nonan['Neighborhood'], Toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

## Examining our Clusters

#### Cluster 1

In [29]:
Toronto_merged_nonan.loc[Toronto_merged_nonan['Cluster Labels'] == 0, Toronto_merged_nonan.columns[[1] + list(range(5, Toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Nail Salon,Grocery Store,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
2,Downtown Toronto,0.0,Coffee Shop,Park,Pub,Bakery,Café,Restaurant,Farmers Market,Sushi Restaurant,Beer Store,Performing Arts Venue
3,North York,0.0,Clothing Store,Furniture / Home Store,Miscellaneous Shop,Accessories Store,Boutique,Vietnamese Restaurant,Coffee Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant
4,Queen's Park,0.0,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Salad Place,Mexican Restaurant,Burger Joint,Café,Sandwich Place,Restaurant
7,North York,0.0,Japanese Restaurant,Baseball Field,Caribbean Restaurant,Gym,Café,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,0.0,Coffee Shop,Café,Sandwich Place,Hotel,Japanese Restaurant,Bank,Gym,Asian Restaurant,Restaurant,Sushi Restaurant
98,Etobicoke,0.0,River,Pool,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant
99,Downtown Toronto,0.0,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Gay Bar,Restaurant,Indian Restaurant,Burrito Place,Gym,Fast Food Restaurant,Mediterranean Restaurant
100,East Toronto Business,0.0,Yoga Studio,Brewery,Garden,Light Rail Station,Spa,Fast Food Restaurant,Smoke Shop,Farmers Market,Skate Park,Auto Workshop


#### Cluster 2

In [30]:
Toronto_merged_nonan.loc[Toronto_merged_nonan['Cluster Labels'] == 1, Toronto_merged_nonan.columns[[1] + list(range(5, Toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,North York,1.0,Park,Yoga Studio,Moroccan Restaurant,Malay Restaurant,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
64,York,1.0,Park,Yoga Studio,Moroccan Restaurant,Malay Restaurant,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant


#### Cluster 3

In [31]:
Toronto_merged_nonan.loc[Toronto_merged_nonan['Cluster Labels'] == 2, Toronto_merged_nonan.columns[[1] + list(range(5, Toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,2.0,Baseball Field,Food Service,Yoga Studio,Moroccan Restaurant,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
101,Etobicoke,2.0,Baseball Field,Yoga Studio,Motel,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


#### Cluster 4

In [32]:
Toronto_merged_nonan.loc[Toronto_merged_nonan['Cluster Labels'] == 3, Toronto_merged_nonan.columns[[1] + list(range(5, Toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3.0,Fast Food Restaurant,Mac & Cheese Joint,Malay Restaurant,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


#### Cluster 5

In [33]:
Toronto_merged_nonan.loc[Toronto_merged_nonan['Cluster Labels'] == 4, Toronto_merged_nonan.columns[[1] + list(range(5, Toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Fast Food Restaurant,Park,Food & Drink Shop,Martial Arts School,Malay Restaurant,Medical Center,Mediterranean Restaurant,Men's Store,Moroccan Restaurant,Metro Station
21,York,4.0,Park,Women's Store,Pool,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Metro Station
35,East York/East Toronto,4.0,Convenience Store,Park,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Yoga Studio
66,North York,4.0,Convenience Store,Park,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Yoga Studio
77,Etobicoke,4.0,Park,Mobile Phone Shop,Sandwich Place,Yoga Studio,Moroccan Restaurant,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
83,Central Toronto,4.0,Tennis Court,Park,Playground,Trail,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
85,Scarborough,4.0,Intersection,Playground,Park,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
91,Downtown Toronto,4.0,Park,Playground,Trail,Yoga Studio,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant


# Exploring New York City

## Neighborhoods of New York City

### Data Collection

We load the json data.

In [34]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [36]:
import json

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [37]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

### Feature Selection

In [38]:
neighborhoods_data = newyork_data['features']

In [39]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### Tranform the data into a pandas dataframe.

In [40]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Geolocations of the New York City Neighborhoods

#### Use geopy library to get the latitude and longitude values of New York City.

In [41]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


## Visualize the Map of New York City

In [42]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [43]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [44]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


## Visualize the Map of Manhattan

In [45]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

# Explore Neighborhoods in Manhattan

Using Foursquare API, we are able to get the venue and venue categories around each neighborhood in Manhattan.

### Venues in Manhattan

Using our previously defined function. Let's get the neaby venues present in each neighborhood of Manhattan.

In [46]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],latitudes=manhattan_data['Latitude'], longitudes=manhattan_data['Longitude'])

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [47]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
1,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Rite Aid,40.875467,-73.908906,Pharmacy
4,Marble Hill,40.876551,-73.91066,Subway,40.874667,-73.909586,Sandwich Place


In [49]:
manhattan_venues.shape

(2996, 7)

We have managed to collect 2996 venue records for the neighborhoods in Manhattan.

### Grouping by Neighborhoods

In [50]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,62,62,62,62,62,62
Carnegie Hill,71,71,71,71,71,71
Central Harlem,37,37,37,37,37,37
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,86,86,86,86,86,86
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [51]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 326 uniques categories.


### Analyze Each Neighborhood

#### One Hot Encoding

We need to Encode our venue categories to get a better result for our clustering.

In [52]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Vietnamese Restaurant,Volleyball Court,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Venue categories mean value

We will group the neighborhoods and calculate the mean of the frequency of occurrence of each category.

In [53]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Vietnamese Restaurant,Volleyball Court,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.00,0.000000,0.014085,0.000000,0.000000,0.000000,0.028169,...,0.000000,0.000000,0.000000,0.028169,0.000000,0.014085,0.056338,0.000000,0.014085,0.028169
2,Central Harlem,0.000000,0.000000,0.00,0.081081,0.054054,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.070000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.020000,0.000000
4,Chinatown,0.000000,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000
5,Civic Center,0.000000,0.000000,0.00,0.000000,0.011628,0.011628,0.000000,0.011628,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.023256,0.000000,0.000000,0.023256
6,Clinton,0.000000,0.000000,0.00,0.000000,0.050000,0.000000,0.000000,0.010000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.010000,0.010000,0.000000,...,0.030000,0.000000,0.000000,0.000000,0.000000,0.040000,0.010000,0.000000,0.000000,0.000000
9,Financial District,0.010000,0.000000,0.00,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.010000,0.010000


### Top venue categories

Reusing our previously defined function to get the top venue categories in the neighborhoods of Manhattan.

In [54]:
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Coffee Shop,Gym,Memorial Site,Food Court,Plaza,Boat or Ferry,Playground,Pizza Place
1,Carnegie Hill,Coffee Shop,Wine Shop,Pizza Place,Cosmetics Shop,Café,Gym,Yoga Studio,Pub,Gym / Fitness Center,Sushi Restaurant
2,Central Harlem,African Restaurant,Sandwich Place,Gym / Fitness Center,French Restaurant,Bar,Seafood Restaurant,Fried Chicken Joint,American Restaurant,Bank,Library
3,Chelsea,Art Gallery,Coffee Shop,Bakery,French Restaurant,American Restaurant,Wine Shop,Italian Restaurant,Ice Cream Shop,Tapas Restaurant,Hotel
4,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Dessert Shop,Salon / Barbershop,Ice Cream Shop,Optical Shop,Noodle House


## Model Building

### K Means

Let's cluster the neighborhoods to roughly 5 to make it easier to analyze.

We use the K Means clustering technique to do so.

In [55]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 1, 1, 1, 1, 0, 1, 1], dtype=int32)

### Labelling Clustered Data

In [56]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Department Store,Supplement Shop,Storage Facility,Steakhouse,Clothing Store,Coffee Shop,Seafood Restaurant,Deli / Bodega,Yoga Studio
1,Manhattan,Chinatown,40.715618,-73.994279,1,Bakery,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Dessert Shop,Salon / Barbershop,Ice Cream Shop,Optical Shop,Noodle House
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Pizza Place,Bakery,Chinese Restaurant,Mobile Phone Shop,Grocery Store,Bank,Park,New American Restaurant,Supplement Shop
3,Manhattan,Inwood,40.867684,-73.921210,2,Mexican Restaurant,Café,Lounge,Bank,Pizza Place,Park,Deli / Bodega,Bakery,Wine Bar,Caribbean Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Café,Sandwich Place,Mexican Restaurant,Bakery,Deli / Bodega,Coffee Shop,Cocktail Bar,Caribbean Restaurant,Indian Restaurant


### Visualizing the Clustered neighborhood

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining our Clusters

#### Cluster 1

In [58]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Pizza Place,Café,Sandwich Place,Mexican Restaurant,Bakery,Deli / Bodega,Coffee Shop,Cocktail Bar,Caribbean Restaurant,Indian Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Pizza Place,Steakhouse,Bank,Sandwich Place,Gas Station,Spanish Restaurant,Spa,New American Restaurant
20,Lower East Side,Café,Art Gallery,Bakery,Chinese Restaurant,Pizza Place,Sandwich Place,Diner,Pet Café,Tailor Shop,Filipino Restaurant
25,Manhattan Valley,Yoga Studio,Mexican Restaurant,Pizza Place,Coffee Shop,Indian Restaurant,Café,Hostel,Bubble Tea Shop,Playground,Liquor Store


#### Cluster 2

In [59]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,American Restaurant,Spa,Dessert Shop,Salon / Barbershop,Ice Cream Shop,Optical Shop,Noodle House
8,Upper East Side,Italian Restaurant,Spa,Exhibit,Bakery,Juice Bar,Coffee Shop,Hotel,Yoga Studio,Wine Shop,American Restaurant
9,Yorkville,Italian Restaurant,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Gym,Bar,Diner,Deli / Bodega,Bakery,Indian Restaurant
10,Lenox Hill,Italian Restaurant,Café,Pizza Place,Bank,Sushi Restaurant,Gym,Burger Joint,Coffee Shop,Gym / Fitness Center,Cocktail Bar
12,Upper West Side,Bakery,Italian Restaurant,Café,Wine Bar,Coffee Shop,Cosmetics Shop,Middle Eastern Restaurant,Indian Restaurant,Ice Cream Shop,Pizza Place
14,Clinton,Theater,Italian Restaurant,Coffee Shop,American Restaurant,Sandwich Place,Gym / Fitness Center,Gym,Spa,French Restaurant,Indie Theater
15,Midtown,Hotel,Clothing Store,Theater,Spa,Food Truck,Sushi Restaurant,Coffee Shop,Bakery,Bookstore,Gym
16,Murray Hill,Coffee Shop,Japanese Restaurant,Hotel,Gym / Fitness Center,Spa,Pizza Place,American Restaurant,Taco Place,Sushi Restaurant,Sandwich Place
17,Chelsea,Art Gallery,Coffee Shop,Bakery,French Restaurant,American Restaurant,Wine Shop,Italian Restaurant,Ice Cream Shop,Tapas Restaurant,Hotel
18,Greenwich Village,Italian Restaurant,Clothing Store,Dessert Shop,Sushi Restaurant,Café,Indian Restaurant,Coffee Shop,American Restaurant,Spa,Cosmetics Shop


#### Cluster 3

In [60]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Pizza Place,Bakery,Chinese Restaurant,Mobile Phone Shop,Grocery Store,Bank,Park,New American Restaurant,Supplement Shop
3,Inwood,Mexican Restaurant,Café,Lounge,Bank,Pizza Place,Park,Deli / Bodega,Bakery,Wine Bar,Caribbean Restaurant
5,Manhattanville,Coffee Shop,Bank,Mexican Restaurant,Italian Restaurant,Food Court,Cuban Restaurant,Shipping Store,Bike Trail,Plaza,Food Truck
6,Central Harlem,African Restaurant,Sandwich Place,Gym / Fitness Center,French Restaurant,Bar,Seafood Restaurant,Fried Chicken Joint,American Restaurant,Bank,Library
11,Roosevelt Island,Park,Coffee Shop,Food & Drink Shop,Scenic Lookout,Supermarket,Café,Bus Line,Gym,Gym / Fitness Center,Bubble Tea Shop
13,Lincoln Square,Café,Concert Hall,Plaza,Theater,Performing Arts Venue,Gym / Fitness Center,Bakery,Indie Movie Theater,Park,French Restaurant
26,Morningside Heights,Café,Bookstore,Sandwich Place,American Restaurant,Deli / Bodega,Park,Coffee Shop,Burger Joint,Gaming Cafe,Outdoor Sculpture
28,Battery Park City,Park,Hotel,Coffee Shop,Gym,Memorial Site,Food Court,Plaza,Boat or Ferry,Playground,Pizza Place
35,Turtle Bay,Italian Restaurant,Park,Deli / Bodega,Steakhouse,Coffee Shop,Sushi Restaurant,Seafood Restaurant,French Restaurant,Sandwich Place,Turkish Restaurant
36,Tudor City,Park,Café,Mexican Restaurant,Deli / Bodega,Bank,Gym / Fitness Center,Sushi Restaurant,Thai Restaurant,Dog Run,Restaurant


#### Cluster 4

In [61]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Park,Yoga Studio,Farmers Market,Bar,Fountain,Coffee Shop,Cocktail Bar,Gas Station,Bistro


#### Cluster 5

In [62]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Department Store,Supplement Shop,Storage Facility,Steakhouse,Clothing Store,Coffee Shop,Seafood Restaurant,Deli / Bodega,Yoga Studio


# Results and Discussion

The neighborhoods of Toronto are very multicultural. There are a lot of different cuisines including Middle Eastern, Moroccan, Latin American, Italian, Japanese, Thai and Chinese. Toronto	 seems to take a step further in this direction by having a lot of restaurants, pub, cafe, coffee shops, sandwich place, breakfast spots, ice cream shop, frozen yogurt shop etc. For leisure, the neighborhoods are set up to have gym, skating rink, lounge, yoga studio, dance studio, playground and monument/landmark. Overall, the city of Toronto offers a multicultural, diverse and certainly an entertaining experience.

New York City has a wide variety of cuisines and eateries including French, African, Japanese, Korean, Chinese etc. There are a lot of hangout spots including café, coffee shop, wine shop and cocktail bars. It has a lot of shopping options too with that of the shopping mall, cosmetics shop, bakery, optical shop etc. For leisure and sight-seeing, there are a lot of parks, playground, yoga studio, gym/fitness center, spa, memorial sites and art gallery. Overall, New York City seems like the relaxing vacation spot with a mix of parks, historic spots and a wide variety of cuisines to try out.

# Conclusion

The purpose of this project was to explore the cities of Toronto and New York City and see how attractive it is to potential tourists and migrants. We explored both the cities based on their postal codes and then extrapolated the common venues present in each of the neighborhoods. Finally, we conclude with clustering similar neighborhoods together.

We could see that each of the neighborhoods in both the cities have a wide variety of experiences to offer which is unique in its own way. The cultural diversity is the evident which also gives the feeling of a sense of inclusion.
Both Toronto and New York City seem to offer a vacation stay or a romantic gateway with a lot of places to explore, beautiful landscapes and a wide variety of culture. In a nutshell, it depends on the stakeholders to decide which experience they would prefer more, and which would more to their liking.
